In [22]:
%run ../../src/feature_selection_scripts/Feature_Selection_Bin_Cla_SRV.ipynb

['Seq', 'sTos', 'sTtl', 'Offset', 'sMeanPktSz', 'dMeanPktSz', 'AckDat', 'icmp', 'tcp', 'udp', 'ECO', 'FIN', 'INT', 'RST', 'Start']


In [23]:
import pickle
from sklearn.ensemble import VotingClassifier
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report


In [24]:
with open('../AdaBoost/adaboost_clf.pkl', 'rb') as f:
    adaboost_clf = pickle.load(f)

with open('../Light_GBM/lgb_clf.pkl', 'rb') as f:
    lgb_clf = pickle.load(f)

with open('../XGBoost/xgb_clf.pkl', 'rb') as f:
    xgb_clf = pickle.load(f)

In [25]:
X = df[selected_feature_labels].values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [26]:
classifiers = [('AdaBoost', adaboost_clf), 
               ('XGBoost', xgb_clf), 
               ('LightGBM', lgb_clf)]

voting_clf = VotingClassifier(estimators=classifiers, voting='hard')

voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)

/opt/homebrew/lib/python3.11/site-packages/sklearn/ensemble/_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[LightGBM] [Info] Number of positive: 553765, number of negative: 358152
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023596 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1338
[LightGBM] [Info] Number of data points in the train set: 911917, number of used features: 15
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.607254 -> initscore=0.435783
[LightGBM] [Info] Start training from score 0.435783


In [27]:
accuracy = accuracy_score(y_test, y_pred)
precision_benign = precision_score(y_test, y_pred, pos_label=0)
recall_benign = recall_score(y_test, y_pred, pos_label=0)
f1_benign = f1_score(y_test, y_pred, pos_label=0)
precision_malicious = precision_score(y_test, y_pred, pos_label=1)
recall_malicious = recall_score(y_test, y_pred, pos_label=1)
f1_malicious = f1_score(y_test, y_pred, pos_label=1)

# Print evaluation metrics
print("Accuracy:", accuracy)
print("precision_benign:", precision_benign)
print("recall_benign:", recall_benign)
print("f1_benign:", f1_benign)
print("precision_malicious:", precision_benign)
print("recall_malicious:", recall_benign)
print("f1_malicious:", f1_malicious)

Accuracy: 0.9996578643497942
precision_benign: 0.9993062926776267
recall_benign: 0.999824392691391
f1_benign: 0.9995652755484216
precision_malicious: 0.9993062926776267
recall_malicious: 0.999824392691391
f1_malicious: 0.9997179385648497
